In [1]:
import datetime
import json
import os
from typing import Annotated, Optional, TypedDict

import markdown
import pandas as pd
import pymysql
import requests
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph

llm = ChatOpenAI(model="gpt-4o-mini")
version = "2024-09-03"

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "LLM_commnet"

In [2]:
connection = pymysql.connect(
    db=os.environ["db_name"],
    user=os.environ["db_user"],
    passwd=os.environ["db_pass"],
    host=os.environ["db_host"],
    port=3306,
    cursorclass=pymysql.cursors.DictCursor,
)
cursor = connection.cursor()
query = """SELECT hd,ca,efa,efb,ranking FROM fp_IPEDS.latest_information"""
cursor.execute(query)
row = cursor.fetchone()
# connection.commit()
hd_year = row["hd"]
efa_year = row["efa"]
rank_year = row["ranking"]
major_year = row["ca"]
students_year = row["efa"] + 1
age_year = row["efb"] + 1
cursor.close()
connection.close()

In [3]:
rank_admin_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/ranking_admin_20231213.php?name="
information_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/school_information_20240821.php?name="
majors_data_url = (
    "https://www.forwardpathway.com/d3v7/dataphp/chatbot/degrees.php?name="
)
SAT_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/score10_20231213.php?name="
students_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/student_comp_20240118.php?name="
age_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/age_mf_20240118.php?name="
international_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/international_students_20240118.php?name="
crime_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/crime_yearly_20240324.php?name="
nearby_data_url = (
    "https://www.forwardpathway.com/d3v7/dataphp/chatbot/school_nearby.php?name="
)
finance_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/finance_yearly_20240118.php?name="
salary_data_url = "https://www.forwardpathway.com/d3v7/dataphp/school_database/staff_salary_20240118.php?name="

---------------排名录取率--------------------

In [4]:
def rank_admin_comments(state):
    try:
        college_postid = state["college_postid"]
        college_unitid = state["college_unitid"]
        college_name = state["college_name"]
        college_cname = state["college_cname"]
        comment_string = state["comment_string"]

        if college_postid == "colleges":
            rank_admin_data = pd.read_json(
                rank_admin_data_url + college_postid + "&ID=" + str(college_unitid)
            )
        else:
            rank_admin_data = pd.read_json(rank_admin_data_url + str(college_postid))
        rank_admin_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """下面给出的是{}({})""".format(college_name, college_cname)
                    + """排名与录取相关数据，数据中'year'为年份，'rank'为USNewws美国大学排名，'rate'为男生录取率,'rate2'为女生录取率,\
                'enroll'为当年入学人数,'deny'为当年拒绝人数,'defer'为当年录取但延迟入学人数，"""
                    + comment_string,
                ),
                ("human", "排名与录取数据如下：{data}"),
            ]
        )
        rank_admin_chain = rank_admin_prompt | llm | StrOutputParser()
        response = rank_admin_chain.invoke({"data": rank_admin_data}).replace(
            "\n\n", "\n"
        )
        rank_admin_comments = markdown.markdown(response)
        return {"rank_admin": rank_admin_comments}
    except:
        return {"rank_admin": None}

------------------------基础数据-------------------------------

In [5]:
def information_comments(state):
    try:
        college_postid = state["college_postid"]
        college_unitid = state["college_unitid"]
        college_name = state["college_name"]
        college_cname = state["college_cname"]
        comment_string = state["comment_string"]
        if college_postid == "colleges":
            reqeust_response = requests.get(
                information_data_url + college_postid + "&ID=" + str(college_unitid)
            )
        else:
            reqeust_response = requests.get(information_data_url + str(college_postid))
        information_data = json.loads(reqeust_response.content)
        information_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """下面给出的是{}({})""".format(college_name, college_cname)
                    + """的相关数据，数据中'type'为学校类型，1为公立，2为私立
                            'tuition'为学费数据，其中'year'为年份，'tuition_in_under'为州内本科学费,'tuition_out_under'为外州本科生学费，'tuition_in_grad'为州内研究生学费，'tuition_out_grad'为外州研究生学费；
                            'students'为学生人数数据，其中'year'为年份，'students_under'为该校本科生人数，'students_grad'为该校研究生人数；
                            'm2w'为男女比例数据，其中'year'为年份，'m2w_men_under'为本科生男生占比，'m2w_women_under'为本科生女生占比，'m2w_men_grad'为研究生男生占比，'m2w_women_grad'为研究生女生占比；
                            'graduation'为毕业率数据，其中'year'为年份，'graduation_100_under'为本科生按时毕业率，'graduation_150_under'为本科生150%时间毕业率；
                            'retention'为学生保有率数据，其中'year'为年份，'retention_under'为学生保有率；
                            's2f'为学生教授比，其中'year'为年份，'s2f_under'为学生教授比；
                            选取其中最重要或者变化比较大3-4个数据做讨论，不一定要所有点都提到"""
                    + comment_string,
                ),
                ("human", "数据如下：{data}"),
            ]
        )
        information_chain = information_prompt | llm | StrOutputParser()
        response = information_chain.invoke({"data": information_data})
        information_comments = markdown.markdown(response).replace("\n", "")
        return {"information": information_comments}
    except:
        return {"information": None}

-----------------------------专业数据---------------------------------

In [6]:
def majors_comments(state):
    try:
        college_postid = state["college_postid"]
        college_unitid = state["college_unitid"]
        college_name = state["college_name"]
        college_cname = state["college_cname"]
        comment_string = state["comment_string"]
        if college_postid == "colleges":
            majors_data = pd.read_json(
                majors_data_url + college_postid + "&ID=" + str(college_unitid),
                orient="index",
                convert_axes=False,
            ).reset_index(drop=True)
        else:
            majors_data = pd.read_json(
                majors_data_url + str(college_postid), orient="index", convert_axes=False
            ).reset_index(drop=True)
        majors_data = majors_data.iloc[
            majors_data[["bachelor", "master", "doctorate"]]
            .sum(axis=1)
            .sort_values(ascending=False)
            .index
        ]
        majors_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """下面给出的是{}年{}({})""".format(
                        major_year, college_name, college_cname
                    )
                    + """的专业毕业人数相关数据，其中associate为副学士学位，bachelor为本科学位，master为硕士学位，doctorate为博士学位，
                            associate_i为副学士学位国际留学生，bachelor_i为本科学位国际留学生，master_i为硕士学位国际留学生，doctorate_i为博士学位国际留学生，
                            学位后面的数字为毕业年份，该栏对应数字为该专业该年份毕业的人数，
                            从数据中找几个重要的或者有特点的类别分析，"""
                    + comment_string,
                ),
                ("human", "数据如下：{data}"),
            ]
        )
        majors_chain = majors_prompt | llm | StrOutputParser()
        response = majors_chain.invoke({"data": majors_data})
        majors_comments = markdown.markdown(response).replace("\n", "")
        return {"majors": majors_comments}
    except:
        return {"majors": None}

----------------------------------SAT/ACT--------------------------------------

In [7]:
def SAT_comments(state):
    try:
        college_postid = state["college_postid"]
        college_unitid = state["college_unitid"]
        college_name = state["college_name"]
        college_cname = state["college_cname"]
        comment_string = state["comment_string"]
        if college_postid == "colleges":
            SAT_data = pd.read_json(
                SAT_data_url + college_postid + "&ID=" + str(college_unitid)
            )
        else:
            SAT_data = pd.read_json(SAT_data_url + str(college_postid))
        SAT_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """下面给出的是{}({})""".format(college_name, college_cname)
                    + """的相关数据，第一层为不同年份的数据，下面分为per和score部分，
                其中per部分包含SAT和ACT成绩提交的人数（'number'）和占录取人数的比例（'per'），
                score部分包含'SATR'为SAT阅读成绩，'SATM'为SAT数学成绩，'ACTC'为ACT综合成绩，'ACTE'为ACT英语成绩，'ACTM'为ACT数学成绩，
                'start'是录取学生中25%该项成绩，'end'是录取学生总75%该项成绩，"""
                    + comment_string,
                ),
                ("human", "数据如下：{data}"),
            ]
        )
        SAT_chain = SAT_prompt | llm | StrOutputParser()
        response = SAT_chain.invoke({"data": SAT_data})
        SAT_comments = markdown.markdown(response).replace("\n", "")
        return {"SAT_ACT": SAT_comments}
    except:
        return {"SAT_ACT": None}

------------------------学生组成---------------------------------

In [8]:
def students_comments(state):
    try:
        college_postid = state["college_postid"]
        college_unitid = state["college_unitid"]
        college_name = state["college_name"]
        college_cname = state["college_cname"]
        comment_string = state["comment_string"]
        if college_postid == "colleges":
            students_data = pd.read_json(
                students_data_url + college_postid + "&ID=" + str(college_unitid)
            )
        else:
            students_data = pd.read_json(students_data_url + str(college_postid))
        students_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """下面给出的是{}年{}({})""".format(
                        students_year, college_name, college_cname
                    )
                    + """的学生组成相关数据，其中'name'是学生分类，有以下几种：'uf'是本科新生，'uj'是本科老生，'ut'是本科转学生，'gr'是研究生，
                'value'是该类学生的人数，'ratioM'是该类学生男生占比，'ratioW'是该类学生女生占比，第一项的'year'是数据的年份
                subs是该类学生的细分人种，有'wh'是白人，'as'是亚裔，'la'是拉丁裔，'pa'是太平洋岛国及其他，'af'是非裔，'nr'是留学生，学生人数代码依旧为'value'，男女比例代码也与上一层相同，在回答中请不要提及各个分类的代码，仅使用真实名称，"""
                    + comment_string,
                ),
                ("human", "数据如下：{data}"),
            ]
        )
        students_chain = students_prompt | llm | StrOutputParser()
        response = students_chain.invoke({"data": students_data})
        students_comments = markdown.markdown(response).replace("\n", "")
        # students_comments=response
        return {"stduents_comp": students_comments}
    except:
        return {"stduents_comp": None}

-------------------学生年龄分布--------------------------------

In [9]:
def age_comments(state):
    try:
        college_postid = state["college_postid"]
        college_unitid = state["college_unitid"]
        college_name = state["college_name"]
        college_cname = state["college_cname"]
        comment_string = state["comment_string"]
        if college_postid == "colleges":
            age_data = pd.read_json(
                age_data_url + college_postid + "&ID=" + str(college_unitid)
            )
        else:
            age_data = pd.read_json(age_data_url + str(college_postid))
        age_data[["underf", "gradf", "totalf"]] = -age_data[["underf", "gradf", "totalf"]]
        age_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """下面给出的是{}年{}({})""".format(
                        age_year, college_name, college_cname
                    )
                    + """的学生年龄组成相关数据，其中'cat'为不同年龄段的分类，'underm'是本科男生，'underf'是本科女生，'gradm'是研究生男生，'gradf'是研究生女生，
                'totalm'是所有学生的那声，'totalf'是所有学生的女生，"""
                    + comment_string,
                ),
                ("human", "数据如下：{data}"),
            ]
        )
        age_chain = age_prompt | llm | StrOutputParser()
        response = age_chain.invoke({"data": age_data})
        age_comments = markdown.markdown(response).replace("\n", "")
        return {"age": age_comments}
    except:
        return {"age": None}

-------------------国际学生数量-----------------------------

In [10]:
def international_comments(state):
    try:
        college_postid = state["college_postid"]
        college_unitid = state["college_unitid"]
        college_name = state["college_name"]
        college_cname = state["college_cname"]
        comment_string = state["comment_string"]
        if college_postid == "colleges":
            international_data = pd.read_json(
                international_data_url + college_postid + "&ID=" + str(college_unitid)
            )
        else:
            international_data = pd.read_json(international_data_url + str(college_postid))
        international_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """下面给出的是{}({})""".format(college_name, college_cname)
                    + """的国际留学生相关数据，'year'为数据年份，其中'undertotal'为该校本科总学生数量，'under'为留学生数量，'underper'为本科学生中留学生占比，
                'gradtotal'为该校研究生总学生数量，'grad'为研究生中留学生数量，'gradper'为研究生中留学生占比"""
                    + comment_string,
                ),
                ("human", "数据如下：{data}"),
            ]
        )
        international_chain = international_prompt | llm | StrOutputParser()
        response = international_chain.invoke({"data": international_data})
        international_comments = markdown.markdown(response).replace("\n", "")
        return {"international": international_comments}
    except:
        return {"international":None}

-------------------------犯罪率---------------------------------------

In [11]:
def crime_comments(state):
    try:
        college_postid = state["college_postid"]
        college_unitid = state["college_unitid"]
        college_name = state["college_name"]
        college_cname = state["college_cname"]
        comment_string = state["comment_string"]
        if college_postid == "colleges":
            crime_data = pd.read_json(
                crime_data_url + college_postid + "&ID=" + str(college_unitid)
            )
        else:
            crime_data = pd.read_json(crime_data_url + str(college_postid))
        crime_dict = {
            "WA": "持枪逮捕",
            "DA": "毒品逮捕",
            "LA": "酗酒逮捕",
            "WD": "持枪记过",
            "DD": "毒品记过",
            "LD": "酗酒记过",
            "DoV": "家暴",
            "DaV": "约会犯罪",
            "S": "跟踪",
            "M": "谋杀",
            "NM": "过失杀人",
            "Ra": "强奸",
            "F": "性扰",
            "I": "乱伦",
            "Ro": "抢劫",
            "AA": "袭击",
            "B": "盗窃",
            "VT": "偷车",
            "A": "纵火",
            "RHF": "宿舍火灾",
        }
        crime_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """下面给出的是{}({})""".format(college_name, college_cname)
                    + """的校园记过、犯罪相关数据，'year'为每组数据的年份，'crime_total'可以忽略，'avg1000'为每1000学生记过、犯罪率，
                'subdata'为细分项，项目名称代码转换为真实名称如下表：{crime_dict}，其中'numberR'为数量，忽略'number'项，在回答最后附上犯罪率的计算方法链接：'https://www.forwardpathway.com/33447'，"""
                    + comment_string,
                ),
                ("human", "数据如下：{data}"),
            ]
        )
        crime_chain = crime_prompt | llm | StrOutputParser()
        response = crime_chain.invoke({"data": crime_data, "crime_dict": crime_dict})
        crime_comments = markdown.markdown(response).replace("\n", "")
        return {"crime": crime_comments}
    except:
        return {"crime": None}

------------------------周边名校------------------------------------

In [12]:
def nearby_comments(state):
    try:
        college_postid = state["college_postid"]
        college_unitid = state["college_unitid"]
        college_name = state["college_name"]
        college_cname = state["college_cname"]
        language_string = state["language_string"]
        fpus = state["fpus"]
        nearby_url = nearby_data_url
        if college_postid == "colleges":
            nearby_url = nearby_url + college_postid + "&ID=" + str(college_unitid)
        else:
            nearby_url = nearby_url + str(college_postid)
        if fpus == "fpus":
            nearby_url = nearby_url + "&v=fpus"
            url_base = "https://www.forwardpathway.us"
        else:
            url_base = "https://www.forwardpathway.com"
    
        nearby_data = pd.read_json(nearby_url)
        nearby_data = (
            nearby_data.sort_values("distance").drop(columns=["distance"]).head(20)
        )
        nearby_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """下面给出的是{}({})""".format(college_name, college_cname)
                    + """的周边名校，数据中'name'为英文名，'cname'为中文名，'rank'为"""
                    + str(rank_year)
                    + """USNews排名，'type'为学校类型，1为综合大学，2为文理学院，postid为学校url链接路径，
                对该校周边名校中挑选重点做一个简单的介绍，提到的学校名上面需要需要超链接，链接名为学校名称，链接url为'"""
                    + url_base
                    + """/postid',其中postid部分请替换为数据中的postid。"""
                    + language_string,
                ),
                ("human", "数据如下：{data}"),
            ]
        )
        nearby_chain = nearby_prompt | llm | StrOutputParser()
        response = nearby_chain.invoke({"data": nearby_data})
        nearby_comments = markdown.markdown(response).replace("\n", "")
        return {"nearby": nearby_comments}
    except:
        return {"nearby": None}

---------------------------财政收入、支出----------------------------------

In [13]:
def finance_convert(x, type):
    for item in x:
        if item["type"] == type:
            return item["num"]
    return


def finance_comments(state):
    try:
        college_postid = state["college_postid"]
        college_unitid = state["college_unitid"]
        college_name = state["college_name"]
        college_cname = state["college_cname"]
        comment_string = state["comment_string"]
        if college_postid == "colleges":
            data = pd.read_json(
                finance_data_url + college_postid + "&ID=" + str(college_unitid)
            )
        else:
            data = pd.read_json(finance_data_url + str(college_postid))
        revenue_array = [
            "Year",
            "学杂费",
            "联邦政府拨款",
            "州政府拨款",
            "地方政府拨款",
            "政府补助、外包合同",
            "私人礼物、赠款、合同",
            "投资回报",
            "资产收入",
            "教育活动收入",
            "其他收入",
            "附属企业收入",
            "附属医院收入",
            "独立活动收入",
        ]
        revenue_df = pd.DataFrame(columns=revenue_array)
        revenue_df["Year"] = data["year"]
        for x in revenue_array[1:]:
            revenue_df[x] = data["revenue"].apply(finance_convert, type=x)
        revenue_df.dropna(axis=1, inplace=True)
        revenue_df["总收入"] = revenue_df.sum(axis=1)
        expense_array = [
            "Year",
            "教学活动",
            "研究活动",
            "公共服务",
            "学术支持",
            "学生服务",
            "学院支持",
            "奖学金支出",
            "其他支出",
            "附属企业支出",
            "附属医院支出",
            "独立活动支出",
        ]
        expense_df = pd.DataFrame(columns=expense_array)
        expense_df["Year"] = data["year"]
        for x in expense_array[1:]:
            expense_df[x] = data["expense"].apply(finance_convert, type=x)
        expense_df.dropna(axis=1, inplace=True)
        expense_df["总支出"] = expense_df.sum(axis=1)
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """下面给出的是{}({})""".format(college_name, college_cname)
                    + """的财政收支数据,数据单位为美元，财政收入数据需要特别注意投资回报为负数的年份，计算收入占比可以用当年分类数值除以总收入数值，
                财政支出数据计算支出占比可以用当年的分类数值除以总支出数值，财政数字尽量使用万，亿等单位，"""
                    + comment_string,
                ),
                ("human", "财政收入数据如下：{revenue}\n\n财政支出数据如下：{expense}"),
            ]
        )
        chain = prompt | llm | StrOutputParser()
        response = chain.invoke({"revenue": revenue_df, "expense": expense_df})
        finance_comments = markdown.markdown(response).replace("\n", "")
        return {"finanace": finance_comments}
    except:
        return {"finanace": None}

--------------------------------教职工工资-----------------------------------------------

In [14]:
def salary_convert(x, typeR, typeS):
    for item in x:
        if item["r"] == typeR:
            return item[typeS]
    return


def salary_comments(state):
    try:
        college_postid = state["college_postid"]
        college_unitid = state["college_unitid"]
        college_name = state["college_name"]
        college_cname = state["college_cname"]
        comment_string = state["comment_string"]
        salary_dict = {
            1: "教授",
            2: "副教授",
            3: "助理教授",
            4: "讲师 (Instructor)",
            5: "讲师 (Lecturer)",
            6: "无职称",
            7: "所有教职工",
        }
        if college_postid == "colleges":
            data = pd.read_json(
                salary_data_url + college_postid + "&ID=" + str(college_unitid)
            )
        else:
            data = pd.read_json(salary_data_url + str(college_postid))
        df = pd.DataFrame()
        df["Year"] = data.year
        for key, item in salary_dict.items():
            df[item + "(平均)"] = data["data"].apply(salary_convert, typeR=key, typeS="t")
            df[item + "（男）"] = data["data"].apply(salary_convert, typeR=key, typeS="m")
            df[item + "（女）"] = data["data"].apply(salary_convert, typeR=key, typeS="w")
        df.dropna(axis=1, inplace=True)
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """下面给出的是{}({})""".format(college_name, college_cname)
                    + """的教职工工资数据，其中'year'为年份，单位为美元，"""
                    + comment_string,
                ),
                ("human", "数据如下：{data}"),
            ]
        )
        chain = prompt | llm | StrOutputParser()
        response = chain.invoke({"data": df})
        salary_comments = markdown.markdown(response).replace("\n", "")
        return {"salary": salary_comments}
    except:
        return {"salary": None}

--------------------------------简介-------------------------------------

In [15]:
def intro_info(state):
    college_postid = state["college_postid"]
    college_unitid = state["college_unitid"]
    college_name = state["college_name"]
    college_cname = state["college_cname"]
    language_string = state["language_string"]
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """对下面给出的学校中文名和英文名，结合你了解的情况，对该学校做一个500字左右最多两个段落的简单综合介绍，可以从下面的几个方面随机选取3-4个方面介绍学校，\
                可以选择的方面如下：特点、校训、人文环境、校园文化、学校设施、师资配置、地理位置、毕业生就业、专业、体育运动、学生生活、杰出校友、最近的热点新闻等等，内容不可以虚构必须是事实，内容尽量不要提及任何负面信息。"""
                + language_string,
            ),
            ("human", "中文名如下：{college_cname}，英文名如下：{college_name}"),
        ]
    )
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke(
        {"college_cname": college_cname, "college_name": college_name}
    )
    intro_comments = markdown.markdown(response).replace("\n", "")
    return {"intro": intro_comments}

--------------------------------get colleges list----------------------------------

In [16]:
def get_colleges_list(post_type="postid"):
    connection = pymysql.connect(
        db=os.environ["db_name"],
        user=os.environ["db_user"],
        passwd=os.environ["db_pass"],
        host=os.environ["db_host"],
        port=3306,
        cursorclass=pymysql.cursors.DictCursor,
    )
    cursor = connection.cursor()
    if post_type == "unitid":
        query = """SELECT "colleges" as postid, tfp.postid as fpus_id,t1.UNITID as unitid,t1.INSTNM as name,t3.translation as cname,NULL as type FROM fp_IPEDS.hd{} t1
JOIN fpus_colleges.transform tfp ON tfp.unitid=t1.UNITID
JOIN fp_IPEDS.EFA t2 ON t2.UNITID=t1.UNITID AND t2.EFALEVEL=1 AND t2.Year={} AND t2.EFTOTLT>=500
LEFT JOIN fp_IPEDS.inst_trans t3 ON t3.UNITID=t1.UNITID
WHERE t1.UNITID NOT IN (SELECT unitid FROM fp_ranking.colleges) AND t1.UNITID NOT IN (SELECT unitid FROM fp_chatGPT.data_comments_en WHERE version="{}")""".format(
            hd_year, efa_year, version
        )
    else:
        query = """SELECT t1.postid,t2.postid as fpus_id,t1.unitid,t1.name,t1.cname,t1.type FROM fp_ranking.colleges t1 LEFT JOIN fpus_colleges.transform t2 ON t2.unitid=t1.unitid WHERE t1.type IN (1,2)"""
    cursor.execute(query)
    rows = cursor.fetchall()
    colleges = pd.DataFrame(
        columns=["postid", "fpus_id", "unitid", "name", "cname", "type"]
    )
    for row in rows:
        colleges = pd.concat([colleges, pd.DataFrame([row])])
    colleges = colleges.reset_index(drop=True)
    cursor.close()
    connection.close()
    return colleges

In [17]:
def initial_college_info_cn(state):
    index = state["index"]
    college_postid = colleges.iloc[index]["postid"]
    college_fpusid = colleges.iloc[index]["fpus_id"]
    college_unitid = colleges.iloc[index]["unitid"]
    college_name = colleges.iloc[index]["name"]
    college_cname = colleges.iloc[index]["cname"]
    comment_string = """对该数据做出一段300字左右文字的简短的评论，标题层级不要超过一级，如果要在输出内容中使用标题的，请使用h3标题，比如<h3>...</h3>。\
可以结合你知道的其他关于该类数据的信息一起评论，可以结合真实的新闻时事信息，但不要捏造信息，遇到比率等需要转换成百分比数。直接给出评论内容，在回答中不要提及数据项的代号，只使用各项数据的真实名称。\
数据中有数值的都为已经发布的实际数据，数据中不含任何预计数据，未发布数据的为'NaN'或者空位，这部分可以忽略。"""
    return {
        "college_postid": college_postid,
        "college_fpusid": college_fpusid,
        "college_unitid": college_unitid,
        "college_name": college_name,
        "college_cname": college_cname,
        "comment_string": comment_string,
        "language_string": "",
        "fpus": None,
    }

In [18]:
def initial_college_info_en(state):
    index = state["index"]
    college_postid = colleges.iloc[index]["postid"]
    college_fpusid = colleges.iloc[index]["fpus_id"]
    college_unitid = colleges.iloc[index]["unitid"]
    college_name = colleges.iloc[index]["name"]
    college_cname = colleges.iloc[index]["cname"]
    language_string = (
        "最终输出结果必须是英文，the final output should in purely English language."
    )
    comment_string = (
        """对该数据做出一段300字左右文字的简短的评论，标题层级不要超过一级，如果要在输出内容中使用标题的，请使用h3标题，比如<h3>...</h3>。\
可以结合你知道的其他关于该类数据的信息一起评论，可以结合真实的新闻时事信息，但不要捏造信息，遇到比率等需要转换成百分比数。直接给出评论内容，在回答中不要提及数据项的代号，只使用各项数据的真实名称。\
数据中有数值的都为已经发布的实际数据，数据中不含任何预计数据，未发布数据的为'NaN'或者空位，这部分可以忽略。"""
        + language_string
    )
    return {
        "college_postid": college_postid,
        "college_fpusid": college_fpusid,
        "college_unitid": college_unitid,
        "college_name": college_name,
        "college_cname": college_cname,
        "comment_string": comment_string,
        "language_string": language_string,
        "fpus": "fpus",
    }

In [19]:
def update_cn(state):
    connection = pymysql.connect(
        db=os.environ["db_name"],
        user=os.environ["db_user"],
        passwd=os.environ["db_pass"],
        host=os.environ["db_host"],
        port=3306,
    )
    cursor = connection.cursor()
    update_sections = [
        "rank_admin",
        "information",
        "majors",
        "SAT_ACT",
        "stduents_comp",
        "age",
        "international",
        "crime",
        "nearby",
        "finanace",
        "salary",
        "intro",
    ]
    postid = state["college_postid"]
    unitid = state["college_unitid"]
    college_name = state["college_name"]
    college_cname = state["college_cname"]
    for update_key in update_sections:
        query = "INSERT INTO fp_chatGPT.data_comments (postid,unitid,type,comment,version) VALUES (%s,%s,%s,%s,%s)"
        update_content = state[update_key]
        if not update_content:
            continue
        if postid == "colleges":
            cursor.execute(
                query,
                (
                    37341,
                    unitid,
                    update_key,
                    update_content,
                    version,
                ),
            )
        else:
            cursor.execute(
                query,
                (
                    postid,
                    unitid,
                    update_key,
                    update_content,
                    version,
                ),
            )
            utc_time_now = datetime.datetime.now(datetime.UTC).strftime(
                "%Y-%m-%d %H:%M:%S"
            )
            time_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            query = "UPDATE fp_forwardpathway.`wp_mmcp_posts` SET `post_modified`=%s,`post_modified_gmt`=%s WHERE `ID`=%s"
            cursor.execute(query, (time_now, utc_time_now, postid))
    connection.commit()
    cursor.close()
    connection.close()
    return

In [20]:
def update_en(state):
    connection = pymysql.connect(
        db=os.environ["db_name"],
        user=os.environ["db_user"],
        passwd=os.environ["db_pass"],
        host=os.environ["db_host"],
        port=3306,
    )
    cursor = connection.cursor()
    update_sections = [
        "rank_admin",
        "information",
        "majors",
        "SAT_ACT",
        "stduents_comp",
        "age",
        "international",
        "crime",
        "nearby",
        "intro",
    ]
    fpusid = state["college_fpusid"]
    unitid = state["college_unitid"]
    college_name = state["college_name"]
    college_cname = state["college_cname"]
    for update_key in update_sections:
        query = "INSERT INTO fp_chatGPT.data_comments_en (postid,unitid,type,comment,version) VALUES (%s,%s,%s,%s,%s)"
        update_content = state[update_key]
        if not update_content:
            continue
        cursor.execute(
            query,
            (
                fpusid,
                unitid,
                update_key,
                update_content,
                version,
            ),
        )
        utc_time_now = datetime.datetime.now(datetime.UTC).strftime("%Y-%m-%d %H:%M:%S")
        time_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        query = "UPDATE fpus_wordpress.`wp_posts` SET `post_modified`=%s,`post_modified_gmt`=%s WHERE `ID`=%s"
        cursor.execute(query, (time_now, utc_time_now, fpusid))
    connection.commit()
    cursor.close()
    connection.close()
    return

In [21]:
class cnState(TypedDict):
    index: int
    comment_string: str
    language_string: str
    college_postid: str
    college_fpusid: str
    college_unitid: str
    college_name: str
    college_cname: str
    fpus: str
    rank_admin: str
    information: str
    majors: str
    SAT_ACT: str
    stduents_comp: str
    age: str
    international: str
    crime: str
    nearby: str
    finanace: str
    salary: str
    intro: str


cn_flow = StateGraph(cnState)
cn_flow.add_node("initial_node", initial_college_info_cn)
cn_flow.add_node("rank_admin_node", rank_admin_comments)
cn_flow.add_node("information_node", information_comments)
cn_flow.add_node("majors_node", majors_comments)
cn_flow.add_node("SAT_node", SAT_comments)
cn_flow.add_node("students_node", students_comments)
cn_flow.add_node("age_node", age_comments)
cn_flow.add_node("international_node", international_comments)
cn_flow.add_node("crime_node", crime_comments)
cn_flow.add_node("nearby_node", nearby_comments)
cn_flow.add_node("finance_node", finance_comments)
cn_flow.add_node("salary_node", salary_comments)
cn_flow.add_node("intro_node", intro_info)
cn_flow.add_node("update_node", update_cn)

cn_flow.add_edge(START, "initial_node")
cn_flow.add_edge("initial_node", "rank_admin_node")
cn_flow.add_edge("initial_node", "information_node")
cn_flow.add_edge("initial_node", "majors_node")
cn_flow.add_edge("initial_node", "SAT_node")
cn_flow.add_edge("initial_node", "students_node")
cn_flow.add_edge("initial_node", "age_node")
cn_flow.add_edge("initial_node", "international_node")
cn_flow.add_edge("initial_node", "crime_node")
cn_flow.add_edge("initial_node", "nearby_node")
cn_flow.add_edge("initial_node", "finance_node")
cn_flow.add_edge("initial_node", "salary_node")
cn_flow.add_edge("initial_node", "intro_node")

cn_flow.add_edge("rank_admin_node", "update_node")
cn_flow.add_edge("information_node", "update_node")
cn_flow.add_edge("majors_node", "update_node")
cn_flow.add_edge("SAT_node", "update_node")
cn_flow.add_edge("students_node", "update_node")
cn_flow.add_edge("age_node", "update_node")
cn_flow.add_edge("international_node", "update_node")
cn_flow.add_edge("crime_node", "update_node")
cn_flow.add_edge("nearby_node", "update_node")
cn_flow.add_edge("finance_node", "update_node")
cn_flow.add_edge("salary_node", "update_node")
cn_flow.add_edge("intro_node", "update_node")

cn_flow.add_edge("update_node", END)

In [22]:
class enState(TypedDict):
    index: int
    comment_string: str
    language_string: str
    college_postid: str
    college_fpusid: str
    college_unitid: str
    college_name: str
    college_cname: str
    fpus: str
    rank_admin: str
    information: str
    majors: str
    SAT_ACT: str
    stduents_comp: str
    age: str
    international: str
    crime: str
    nearby: str
    intro: str


en_flow = StateGraph(enState)
en_flow.add_node("initial_node", initial_college_info_en)
en_flow.add_node("rank_admin_node", rank_admin_comments)
en_flow.add_node("information_node", information_comments)
en_flow.add_node("majors_node", majors_comments)
en_flow.add_node("SAT_node", SAT_comments)
en_flow.add_node("students_node", students_comments)
en_flow.add_node("age_node", age_comments)
en_flow.add_node("international_node", international_comments)
en_flow.add_node("crime_node", crime_comments)
en_flow.add_node("nearby_node", nearby_comments)
en_flow.add_node("intro_node", intro_info)
en_flow.add_node("update_node", update_en)

en_flow.add_edge(START, "initial_node")
en_flow.add_edge("initial_node", "rank_admin_node")
en_flow.add_edge("initial_node", "information_node")
en_flow.add_edge("initial_node", "majors_node")
en_flow.add_edge("initial_node", "SAT_node")
en_flow.add_edge("initial_node", "students_node")
en_flow.add_edge("initial_node", "age_node")
en_flow.add_edge("initial_node", "international_node")
en_flow.add_edge("initial_node", "crime_node")
en_flow.add_edge("initial_node", "nearby_node")
en_flow.add_edge("initial_node", "intro_node")

en_flow.add_edge("rank_admin_node", "update_node")
en_flow.add_edge("information_node", "update_node")
en_flow.add_edge("majors_node", "update_node")
en_flow.add_edge("SAT_node", "update_node")
en_flow.add_edge("students_node", "update_node")
en_flow.add_edge("age_node", "update_node")
en_flow.add_edge("international_node", "update_node")
en_flow.add_edge("crime_node", "update_node")
en_flow.add_edge("nearby_node", "update_node")
en_flow.add_edge("intro_node", "update_node")

en_flow.add_edge("update_node", END)

In [23]:
def reduce_func(left, right):
    return left


class EntryGraphState(TypedDict):
    index: Annotated[int, reduce_func]


def entry_graph(state):
    return


general_flow = StateGraph(EntryGraphState)
general_flow.add_node("graph_entry_node", entry_graph)
general_flow.add_node("cn_commnets", cn_flow.compile())
general_flow.add_node("en_commnets", en_flow.compile())

general_flow.add_edge(START, "graph_entry_node")
general_flow.add_edge("graph_entry_node", "cn_commnets")
general_flow.add_edge("graph_entry_node", "en_commnets")
general_flow.add_edge("cn_commnets", END)
general_flow.add_edge("en_commnets", END)
app = general_flow.compile()

from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

display(
    Image(
        app.get_graph(xray=1).draw_mermaid_png(
            curve_style=CurveStyle.BASIS,
            node_colors=NodeStyles(
                first="fill:#FDFFB6",
                last="fill:#FFADAD",
                default="fill:#CAFFBF,line-height:3",
            ),
            draw_method=MermaidDrawMethod.API,
        ),
        width=2500,
    )
)
img = app.get_graph(xray=1).draw_mermaid_png(
    curve_style=CurveStyle.BASIS,
    node_colors=NodeStyles(
        first="fill:#FDFFB6",
        last="fill:#FFADAD",
        default="fill:#CAFFBF,line-height:1",
    ),
    draw_method=MermaidDrawMethod.API,
)
with open("LLM_data_comments_flow.png", "wb") as png:
    png.write(img)

In [25]:
colleges = get_colleges_list(post_type="postid")

In [26]:
app.invoke({"index": 0})

{'index': 0}